# Puzzle 1: Calculate the highest possible amp signal

### Unique phase setting between 0 and 4 for any amp --> what is the highest signal?

In [1]:
import numpy as np
from copy import deepcopy

## Load input

In [2]:
with open('./input7.txt', 'r') as file:

    software = file.readlines()

#### Convert data to list of integers

In [3]:
software = list(map(int, software[0].split(',')))

## Calculation

In [4]:
def intcode_computer(inputs, data):
    i = 0
    run = True

    while run is True:
        if len(str(data[i])) < 5:
            add = (5-len(str(data[i])))*'0'
            data[i] = '{0}{1}'.format(add, str(data[i]))

        optcode = data[i][-2:]
        mode1 = data[i][-3]
        mode2 = data[i][-4]
        mode3 = data[i][-5]

        if mode1 == '0' and optcode != '99':
            param1 = int(data[data[i+1]])
        else:
            param1 = int(data[i+1])

        if optcode == '01':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])
            data[data[i+3]] = param1 + param2
            i += 4

        if optcode == '02':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])
            data[data[i+3]] = param1 * param2
            i += 4

        if optcode == '03':
            data[data[i+1]] = inputs[0]
            del inputs[0]
            i += 2

        if optcode == '04':
            if mode1 == '0':
                out = data[data[i+1]]
                # print(data[data[i+1]])
            else:
                out = data[data[i+1]]
                # print(data[i+1])
            i += 2

        if optcode == '05':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 != 0:
                i = param2
            else:
                i += 3

        if optcode == '06':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 == 0:
                i = param2
            else:
                i += 3

        if optcode == '07':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 < param2:
                data[data[i+3]] = 1
            else:
                data[data[i+3]] = 0
            i += 4

        if optcode == '08':
            if mode2 == '0':
                param2 = int(data[data[i+2]])
            else:
                param2 = int(data[i+2])

            if param1 == param2:
                data[data[i+3]] = 1
            else:
                data[data[i+3]] = 0
            i += 4

        if optcode == '99':
            run = False
    return out

In [5]:
best_output = 0

for pA in range(0, 5):
    outA = intcode_computer([pA, 0], deepcopy(software))
    for pB in range(0, 5):
        if pA == pB:
            outB = False
        else:
            outB = intcode_computer([pB, outA], deepcopy(software))
        for pC in range(0, 5):
            if pC in [pA, pB] or outB == False:
                outC = False
            else:
                outC = intcode_computer([pC, outB], deepcopy(software))
            for pD in range(0, 5):
                if pD in [pA, pB, pC] or outC == False or outB == False:
                    outD = False
                else:
                    outD = intcode_computer([pD, outC], deepcopy(software))
                for pE in range(0, 5):
                    if pE in [pA, pB, pC, pD] or outD == False or outC == False or outB == False:
                        outE = 0
                    else:
                        outE = intcode_computer([pE, outD], deepcopy(software))
                    
                    if outE > best_output:
                        best_output = outE
                        best_combination = [pA, pB, pC, pD, pE]

In [6]:
print('The highest signal is {0}.'.format(best_output))

The highest signal is 77500.


# Puzzle 2:  Calculate the highest possible amp signal using a feedback loop

### phase settings from 5 to 9

In [11]:
class intcode_computer(object):
    
    def __init__(self, phase, data):
        
        self.phase = phase
        self.data = data
        self.inputs = [phase]
        self.i = 0

    def step(self, amp_in):
        self.inputs.append(amp_in)
        self.run = True
        self.done = False
        self.out = 0

        while self.run is True:
            if len(str(self.data[self.i])) < 5:
                add = (5-len(str(self.data[self.i])))*'0'
                self.data[self.i] = '{0}{1}'.format(add, str(self.data[self.i]))

            optcode = self.data[self.i][-2:]
            mode1 = self.data[self.i][-3]
            mode2 = self.data[self.i][-4]
            mode3 = self.data[self.i][-5]

            if mode1 == '0' and optcode != '99':
                param1 = int(self.data[self.data[self.i+1]])
            else:
                if optcode != '99':
                    param1 = int(self.data[self.i+1])

            if optcode == '01':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])
                self.data[self.data[self.i+3]] = param1 + param2
                self.i += 4

            if optcode == '02':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])
                self.data[self.data[self.i+3]] = param1 * param2
                self.i += 4

            if optcode == '03':
                if len(self.inputs) > 0:
                    self.data[self.data[self.i+1]] = self.inputs[0]
                    del self.inputs[0]
                    self.i += 2
                else:
                    self.run = False
                    # print('Amp waits for input')
                    break
                    
            if optcode == '04':
                if mode1 == '0':
                    self.out = self.data[self.data[self.i+1]]
                else:
                    self.out = self.data[self.data[self.i+1]]
                self.i += 2

            if optcode == '05':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 != 0:
                    self.i = param2
                else:
                    self.i += 3

            if optcode == '06':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 == 0:
                    self.i = param2
                else:
                    self.i += 3

            if optcode == '07':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 < param2:
                    self.data[self.data[self.i+3]] = 1
                else:
                    self.data[self.data[self.i+3]] = 0
                self.i += 4

            if optcode == '08':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                else:
                    param2 = int(self.data[self.i+2])

                if param1 == param2:
                    self.data[self.data[self.i+3]] = 1
                else:
                    self.data[self.data[self.i+3]] = 0
                self.i += 4

            if optcode == '99':
                self.run = False
                self.i += 1
                self.done = True
        return self.out, self.done

#### generate all possible phase settings

In [12]:
from itertools import permutations 

In [13]:
phase_settings = list(permutations(range(5, 10))) 

In [15]:
best_output = 0

for phase in phase_settings:
    done = [False]*5
    outE = 0
    
    ampA = intcode_computer(phase[0], deepcopy(software))
    ampB = intcode_computer(phase[1], deepcopy(software))
    ampC = intcode_computer(phase[2], deepcopy(software))
    ampD = intcode_computer(phase[3], deepcopy(software))
    ampE = intcode_computer(phase[4], deepcopy(software))
    
    while True not in done:
        outA, done[0] = ampA.step(outE)
        outB, done[1] = ampB.step(outA)
        outC, done[2] = ampC.step(outB)
        outD, done[3] = ampD.step(outC)
        outE, done[4] = ampE.step(outD)
        if True in done and outE > best_output:
            best_output = outE

print('The value of the highest possible amp signal is {0}.'.format(best_output))

The value of the highest possible amp signal is 22476942.
